# Table Relations and Normalization

## The SQLite Shell

So far, we've learned lots of skills to extract data from databases. This mission will teach the other side of the equation - how to create and modify databases. Rather than working with a SQL editor, in this mission we'll be working with the **SQLite shell**.<br>

You'll need to have basic knowledge of how to work with the command line - if you haven't already, we recommend completing our [command line beginner course](https://www.dataquest.io/course/command-line-beginner).<br>

We'll be continuing to work with the Chinook music store database, which is stored in the database file `chinook.db`. To launch the SQLite shell, you use the `sqlite3` command followed by the name of the database file as an argument.

```sql
/home/dq$ sqlite3 chinook.db
```

When you launch the SQLite shell, you will be shown the SQLite prompt. Let's see what that looks like when we open our chinook.db file:

In [12]:
%%bash
sqlite3 ./data/chinook.db

If you write a query, you'll get the response displayed in your console. If you press enter, the SQLite prompt will change to `...>` and you can continue writing your query on multiple lines. As a result, unlike any of the places you've written SQL queries so far, including a semicolon (`;`) is necessary in the SQLite shell. If you don't use it, the shell will not know if you are finished writing your query. Let's write a query to look at one of our tables from the Chinook database.

In [84]:
%%bash
sqlite3 ./data/chinook.db
SELECT
track_id,
name,
album_id
FROM track
WHERE album_id = 3;

3|Fast As a Shark|3
4|Restless and Wild|3
5|Princess of the Dawn|3


The first thing you may notice is that we don't have the column names displayed. SQLite has a number of **dot commands** which you can use to help you work with databases. When you use a dot command, you don't need to use a semicolon. One that you'll want to use often is `.headers on`, which switches column headers on. Let's see what the output of our query looks like after we turn column headers on:

In [85]:
%%bash
sqlite3 ./data/chinook.db
.headers on
SELECT track_id, name, album_id
FROM track
WHERE album_id = 3;

track_id|name|album_id
3|Fast As a Shark|3
4|Restless and Wild|3
5|Princess of the Dawn|3


The next thing is that it's hard to read down the columns, since they don't line up. There's another dot command, `.mode`, that will help us out here. The `.mode` dot command allows us to select from a few different display modes. We'll use `.mode column` to allow for easier to read outputs. Here's the output of our query after switching to column mode:

In [86]:
%%bash
sqlite3 ./data/chinook.db
.mode column
SELECT track_id, name, album_id
FROM track
WHERE album_id = 3;

3           Fast As a Shark  3         
4           Restless and Wi  3         
5           Princess of the  3         


There are several other dot commands you'll use often:

* `.help` - Displays help text showing all dot commands and their function.
* `.tables` - Displays a list of all tables and views in the current database.
* `.shell` [command] - Run a command like ls or clear in the system shell.
* `.quit` - Quits the SQLite shell.

* Launch the SQLite shell, connecting to the `chinook.db` database file.
* Use `.tables` to display the tables in the Chinook database.
* Use dot commands to turn headers on, and set the mode to 'column'.
* Write a query on one of the tables in the chinook.db database.
* Display the help text.
* Run `.shell clear` to see what it does.
* Quit the SQLite shell.

In [87]:
%%bash
sqlite3 ./data/chinook.db
.tables
.headers on
.mode column
SELECT track_id, name, album_id FROM track WHERE track_id = 3;
.quit

album                   employee                playlist              
artist                  genre                   playlist_track        
customer                invoice                 track                 
customer_gt_90_dollars  invoice_line            wishlist              
customer_usa            media_type              wishlist_track        
track_id    name             album_id  
----------  ---------------  ----------
3           Fast As a Shark  3         


## Creating Tables

Now that we've learned the basics of the SQLite shell, let's learn how to create a database table. Here's the syntax for `CREATE TABLE`:

```
CREATE TABLE [table_name] (
    [column1_name] [column1_type],
    [column2_name] [column2_type],
    [column3_name] [column3_type],
    [...]
);
```

Each column in SQLite must have a type. While some database systems have as many as 50 distinct data types, SQLite uses only 5 behind the scenes:

* `TEXT`
* `INTEGER`
* `REAL`
* `NUMERIC`
* `BLOB`

If you have any experience with other database systems, you might be familiar with other types such as `VARCHAR`, `REAL`, and `DATETIME`. SQLite accepts most common types in a `CREATE` statement, but behind the scenes will convert them to one of its 5 base types.<br>

The table below shows each of the types, along with examples of data commonly stored in the type, and some 'equivalent' types from other database systems. If you're not familiar with these other types, don't be concerned - we'll cover them in some more detail in a later course.

Type|Commonly Used For|Equivalent Types
---|---|---
`TEXT`|Names<br>Email Addresses<br>Dates and Times<br>Phone Numbers|`CHARACTER`<br>`VARCHAR`<br>`NCHAR`<br>`NVARCHAR`<br>`DATETIME`<br>
`INTEGER`|IDs<br>Quantities|`INT`<br>`SMALLINT`<br>`BIGINT`<br>`INT8`
`REAL`|Weights<br>Averages|`DOUBLE`<br>`FLOAT`
`NUMERIC`|Prices<br>Statuses|`DECIMAL`<br>`BOOLEAN`
`BLOB`|Binary Data|`BLOB`

Just like with views, if you try to create a table that already exists you will get an error. If you make a mistake when you create a table, you can use the `DROP` statement to remove the table so you can create it again:

```
DROP TABLE [table_name];
```
You can also use the SQLite dot command `.schema [table_name]` to view the schema for a table you have just created to check where you might have gone wrong.<br>

To practice, we'll create a new table in a new database file. If you launch the SQLite shell with the argument of a filename that doesn't exist, SQLite will create an empty database that that filename.<br>

Make sure you quit the SQLite shell at the end, as it triggers our answer checking.

* Launch the SQLite shell and create a new database file, `new_database.db`.
* Create a new table, `user`, with the following columns:
  * `user_id`, with type `INTEGER`
  * `first_name`, with type `TEXT`
  * `last_name`, with type `TEXT`
* Use the `.schema` dot command to view the schema for your newly created table.
* Quit the SQLite shell.

In [88]:
%%bash
sqlite3 ./data/new_database.db
DROP TABLE IF EXISTS user;

CREATE TABLE user (user_id INTEGER,
first_name TEXT,
last_name TEXT);
.schema user
.quit

CREATE TABLE user (user_id INTEGER,
first_name TEXT,
last_name TEXT);


## Primary and Foreign Keys

We have been using schema diagrams to identify the relationships between tables. Below is an excerpt of the schema diagram for the Chinook database which shows the relationship between the `invoice` and `invoice_line` tables:

![invoice-invoice-line](https://s3.amazonaws.com/dq-content/192/chinook_pk_fk.svg)

We previously learned that each table has one or more columns shaded in yellow, which indicates they are the **primary key**. A primary key is a unique identifier for each row - you cannot have two rows in a table with the same value for the primary key column(s).<br>

When two tables have a relation, there will be a column in one table that is a primary key in another table. For example, in the `invoice_line` table, the `invoice_id` column is the primary key from the `invoice` table. This is known as a **foreign key**. By defining a foreign key, our database engine will prevent us from adding rows where the foreign key value doesn't exist in the other table, which helps to prevent errors in our data (note that by default SQLite doesn't force foreign key constraints, however we have [changed the default](https://stackoverflow.com/a/44857286/4691920) for this mission).<br>

Usually, a primary key is specified as part of a create statement. Once the primary key is defined, the database engine will prevent any new rows from being added to the database if they have the same primary key as any existing rows. If we wanted to re-create the table from the previous exercise with a primary key, we would use this syntax:

```
CREATE TABLE user (
    user_id INTEGER PRIMARY_KEY,
    first_name TEXT,
    last_name TEXT
);
```


Let's say we wanted to create a new table purchase which tracks basic information about a purchase made by one of our users. Our create statement might look like this:


```sql
CREATE TABLE purchase (
    purchase_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    purchase_date TEXT,
    total NUMERIC,
    FOREIGN KEY (user_id) REFERENCES user(user_id)
);
```

By adding a `FOREIGN KEY` clause, we can define one of our columns as a foreign key and specify the table and column that it references. We're going to use what we've learned about creating tables, primary keys and foreign keys to add new tables to our Chinook database that allow customers to create "wishlists" of tracks they would like to buy.<br>

We'll start by adding a table to store the name of the wishlist and the customer that created the wishlist. The schema is shown below:

![customer-wishlist](https://s3.amazonaws.com/dq-content/192/wishlist_1.svg)

* Launch the SQLite shell, connected to the `chinook.db` database.
* Create a new table, `wishlist`, with the following columns:
  * `wishlist_id`, with type `INTEGER`.
    * This column should be the primary key.
  * `customer_id`, with type `INTEGER`.
    * This column should have a foreign key relationship with the `customer_id` column from the `customer` table.
  * `name`, with type `TEXT`.
* Quit the SQLite shell.

In [89]:
%%bash
sqlite3 ./data/chinook.db
DROP TABLE IF EXISTS wishlist;

CREATE TABLE wishlist (
wishlist_id INTEGER,
customer_id INTEGER,
name TEXT,
PRIMARY KEY (wishlist_id),
FOREIGN KEY (customer_id) REFERENCES customer (customer_id)
);
.quit

## Database Normalization

When we created our `wishlist` table, we didn't include a `track_id` column to store which tracks are in the users wishlist. To understand why, let's take a look at what the table might look like if we stored all the data in a single table.

wishlist_id|customer_id|name|track_id
---|---|---|---
1|34|Joao's awesome wishlist|1158
1|34|Joao's awesome wishlist|2646
1|34|Joao's awesome wishlist|1990
2|18|Amy loves pop|3272
2|18|Amy loves pop|3470

There are are some drawbacks to storing the data this way:

* **Data Duplication** - we are storing the name of each wishlist multiple times.
* **Data Modification** - if we want to change the name of one of the wishlists, we have to modify multiple rows.
* **Data Integrity** - There is nothing to stop a row being added with the wrong wishlist name, and if that happened we wouldn't know which was the correct name.

The process of optimizing the design of databases to minimize these issues is called **database normalization**. In database normalization theory, there are several different phases of normalization, known as [normal forms](http://www.bkent.net/Doc/simple5.htm). Knowing each normal form is not as important as understanding the goals of normalization, and designing your databases to avoid data duplication and integrity issues. We'll learn more about database normalization in the next mission, however for now let's look at how we can design our wishlist tables with normalization in mind:

![design-wishlist](https://s3.amazonaws.com/dq-content/192/wishlist_2.svg)

In addition to the `wishlist` table we made in the previous screen, we have added a new `wishlist_track` table and shown its relationship to the existing `track` table. The `wishlist_track` table has two columns that are both yellow to indicate that they're primary keys, since neither column will uniquely identify each row by itself. When two or more columns combine to form a primary key it is called a compound primary key. To create a **compound primary key**, you use the `PRIMARY KEY` clause:

```
CREATE TABLE [table_name] (
    [column_one_name] [column_one_type],
    [column_two_name] [column_two_type],
    [column_three_name] [column_three_type],
    [column_four_name] [column_four_type],
    PRIMARY KEY (column_one_name, column_two_name)
);
```

Both columns in the `wishlist_track` table also have lines to indicate that they are foreign keys. To create a table with multiple foreign keys, you simply use multiple `FOREIGN KEY` clauses.


* Launch the SQLite shell, connected to the `chinook.db` database.
* Create a new table, `wishlist_track`:
  * The table should have the following columns:
    * `wishlist_id`, with type `INTEGER`.
    * `track_id`, with type `INTEGER`.
  * A primary key should be specified, composed of both columns from the table.
  * Each of the columns should be designated as a foreign key as indicated in the schema diagram.
* Quit the SQLite shell.

In [90]:
%%bash
sqlite3 ./data/chinook.db

DROP TABLE IF EXISTS wishlist_track;

CREATE TABLE wishlist_track (
    wishlist_id INTEGER,
    track_id INTEGER,
    PRIMARY KEY (wishlist_id, track_id),
    FOREIGN KEY (wishlist_id) REFERENCES wishlist (wishlist_id),
    FOREIGN KEY (track_id) REFERENCES track (track_id)
);
.quit

## Inserting and Deleting Rows

Now that we've created the tables to hold our wishlist data, let's add some rows to those tables. To add rows to a SQL table, we'll use the `INSERT` statement:

```
INSERT INTO [table_name] (
    [column1_name],
    [column2_name],
    [column3_name]
) VALUES (
    [value1],
    [value2],
    [value3]
);
```

If you are inserting values into every column in a table, you don't need to list the column names:

```
INSERT INTO [table_name] VALUES ([value1], [value2], [value3]);
```

Additionally, you can insert multiple rows in a single statement:

```
INSERT INTO [table_name]
VALUES
    ([value1], [value2], [value3]),
    ([value4], [value5], [value6]),
    [...]
```

Because of our foreign key constraints, we'll need to start by adding rows to the `wishlist` table, and then add rows to the `wishlist_track` table. If we don't, our insert statement will fail. At the end, we want our `wishlist` table to contain this data:

wishlist_id|customer_id|name
---|---|---
1|34|Joao's awesome wishlist
2|18|Amy loves pop

And our `wishlist+track` table to contain this data:

wishlist_id|track_id
---|---
1|1158
1|2646
1|1990
2|3272
2|3470

If you make an error while inserting new rows, you'll need to use the `DELETE` statement to remove all rows:

```sql
DELETE FROM [table_names]
```

Or use it with a where statement to remove selected rows:

```sql
DELETE FROM [table_name]
WHERE [expression];
```

* Launch the SQLite shell, connected to the `chinook.db` database.
* Add new rows to the `wishlist` table, in order, as shown above.
* Add new rows to the `wishlist_track` table, in order, as shown above.
* Quit the SQLite shell.

In [91]:
%%bash
sqlite3 ./data/chinook.db

INSERT INTO wishlist
(wishlist_id, customer_id, name)
VALUES (1, 34, "Joao's awesome wishlist"),
(2, 18, "Amy loves pop");

INSERT INTO wishlist_track
(wishlist_id, track_id) VALUES 
(1, 1158), (1, 2646), (1, 1990), (2, 3272),(2, 3470);

.quit

## Adding Columns to a Table

We now have two tables to track our wishlist data, and have seeded some data. But what should we do when a customer wants to remove a track from their wishlist? One approach might be just to `DELETE` the row from `wishlist_track`. The downside of this approach is that we don't retain any historical data on which tracks were added to wishlists which reduces our ability to analyze this in the future.<br>

A better approach would be to add a column that has a boolean value to show whether the row is active or not, and just change that value if the user wants to delete a track. We can do a similar thing with the wishlists themselves, so users can delete (or technically, deactivate) wishlists they no longer want to use.<br>

We'll need to add a column to each of our tables. We can use the `ALTER` statement to do this.

```sql
ALTER TABLE [table_name]
ADD COLUMN [column_name] [column_type];
```

As we learned earlier, SQLite supports only five basic types - the closest thing to a boolean type is `NUMERIC`, where the values `1` and `0` will represent true and false respectively. Let's create active columns for both of our wishlist tables.



* Launch the SQLite shell, connected to the `chinook.db` database.
* Create a new column, `active`, in the `wishlist` table with type `NUMERIC`.
* Create a new column, `active`, in the `wishlist_track` table with type `NUMERIC`.
* Quit the SQLite shell.

In [92]:
%%bash
sqlite3 ./data/chinook.db

ALTER TABLE wishlist
ADD COLUMN active NUMERIC;

ALTER TABLE wishlist_track
ADD COLUMN active NUMERIC;

.quit

## Adding Values to Existing Rows

We've added our columns to both wishlist tables, but currently they don't have any data in them. To change values for existing rows, we use the `UPDATE` statement:

```sql
UPDATE [table_name]
SET [column_name] = [expression]
WHERE [expression]
```

The `WHERE` clause is optional, and can contain any expression that would be valid in a `SELECT` statement.<br>

There are several variations we can use for our `SET` clause. First we can use a **single value**:<br>

```sql
UPDATE customer
SET phone = "+55 (12) 3921-4464"
WHERE customer_id = 1
```

We can use a **subquery that returns a single value**:

```sql
UPDATE track
SET unit_price = (
                    SELECT AVG(unit_price)
                    FROM track
                 )
```

We can use a **column, or function on an existing column**:

```sql
UPDATE track
SET unit_price = unit_price * 1.1
```

Lastly, we can **set more than one column at once**:

```sql
UPDATE wishlist_track
SET
    active = 1,
    purchased = 0;
```

Because our active columns will store a `1` for true and `0` for false, we'll set the values to `1` for every row.

* Launch the SQLite shell, connected to the **chinook.db** database.
* Set all values for the **active** column in the `wishlist` table to `1`.
* Set all values for the **active** column in the `wishlist_track` table to `1`.
* Quit the SQLite shell.

In [93]:
%%bash
sqlite3 ./data/chinook.db

UPDATE wishlist SET active = 1;
UPDATE wishlist_track SET active = 1;
.quit

In [94]:
%%bash
sqlite3 ./data/chinook.db
.headers on
.mode columns

SELECT * FROM wishlist;
SELECT * FROM wishlist_track;

wishlist_id  customer_id  name                     active    
-----------  -----------  -----------------------  ----------
1            34           Joao's awesome wishlist  1         
2            18           Amy loves pop            1         
wishlist_id  track_id    active    
-----------  ----------  ----------
1            1158        1         
1            2646        1         
1            1990        1         
2            3272        1         
2            3470        1         


## Challenge
### Adding Sales Tax Capabilities

Let's put everything we've learned into action with a challenge. Up until now, the Chinook store hasn't had to collect sales tax, but due to some new tax legislation, we will need to charge customers in certain countries taxes on their purchases.<br>

We need to add the following columns to the `invoice` table:
* `subtotal`
* `tax`

We also need to update all existing invoices with data for these two new columns. The value for the `tax` column will need to be `0` for every row, and the value for the `subtotal` column will need to be the same as the `total` column for each row.<br>

Here's what the first five rows of those columns will need to look like when you're done:

```bash
sqlite> SELECT
   ...>    invoice_id,
   ...>    subtotal,
   ...>    tax,
   ...>    total
   ...> FROM invoice
   ...> LIMIT 5;
invoice_id  subtotal    tax         total
----------  ----------  ----------  ----------
1           15.84       0           15.84
2           9.9         0           9.9
3           1.98        0           1.98
4           7.92        0           7.92
5           16.83       0           16.83
```

* Launch the SQLite shell, connected to the `chinook.db` database.
* Add two new columns, with values, to the `invoice` table:
  * `tax`, with type `NUMERIC`.
    * The value for all existing rows should be `0`.
  * `subtotal`, with type `NUMERIC`.
    * The value for each row should be the same as that row's value for `total`.
* Quit the SQLite shell.

In [ ]:
%%bash
sqlite3 ./data/chinook.db

ALTER TABLE invoice ADD COLUMN tax NUMERIC;
UPDATE invoice SET tax = 0;

ALTER TABLE invoice ADD COLUMN subtotal NUMERIC;
UPDATE invoice SET subtotal = total + tax;
.quit

In [96]:
%%bash
sqlite3 ./data/chinook.db
.headers on
.mode columns

SELECT invoice_id, subtotal, tax, total FROM invoice LIMIT 5;
.quit

invoice_id  subtotal    tax         total     
----------  ----------  ----------  ----------
1           15.84       0           15.84     
2           9.9         0           9.9       
3           1.98        0           1.98      
4           7.92        0           7.92      
5           16.83       0           16.83     


## Next Steps

In this mission, we learned:
* How to work with the SQLite shell.
* How to create new tables and assign primary and foreign keys.
* Basic concepts of database normalization.
* How to insert new rows into tables.
* How to add new columns to existing tables.
* How to update existing data in tables.

In the final guided project of this course, we'll put these new skills into practice by building a normalized database and importing data.